In [1]:
import qualified Data.Graph as G
import Control.Monad.ST
import Data.STRef

import qualified Data.Map.Strict    as M
import qualified Data.Set           as S
import qualified Data.IntMap.Strict as I
import qualified Data.Array         as A

In [2]:
import Control.Monad (forM_, when)
import Data.Maybe (isNothing, fromJust)

tarjan graph = runST $ do
    index    <- newSTRef 0
    stack    <- newSTRef []
    indices  <- newSTRef I.empty
    lowlinks <- newSTRef I.empty
    output   <- newSTRef []
    
    forM_ (G.vertices graph) $ \v -> do
        vIndex <- I.lookup v <$> readSTRef indices
        when (isNothing vIndex) $ strongConnect v graph index stack indices lowlinks output
    
    reverse <$> readSTRef output

strongConnect v graph index stack indices lowlinks output = do
    i <- readSTRef index
    modifySTRef' indices  (I.insert v i)
    modifySTRef' lowlinks (I.insert v i)
    modifySTRef' index (+1)
    push stack v

    forM_ (graph A.! v) $ \w -> do
        wIndex <- I.lookup w <$> readSTRef indices
        if isNothing wIndex
            then do
                strongConnect w graph index stack indices lowlinks output
                vLowLink <- fromJust . I.lookup v <$> readSTRef lowlinks
                wLowLink <- fromJust . I.lookup w <$> readSTRef lowlinks
                modifySTRef' lowlinks (I.insert v $ min vLowLink wLowLink)
            else do
                wOnStack <- elem w <$> readSTRef stack
                when wOnStack $ do
                    vLowLink <- fromJust . I.lookup v <$> readSTRef lowlinks
                    modifySTRef' lowlinks (I.insert v $ min vLowLink (fromJust wIndex))

    vLowLink <- fromJust . I.lookup v <$> readSTRef lowlinks
    vIndex   <- fromJust . I.lookup v <$> readSTRef indices
    when (vLowLink == vIndex) $ do
        scc <- addSCC v [] stack
        modifySTRef' output (scc:)    

addSCC v scc stack = do
    w <- pop stack
    let scc' = w:scc
    if w == v then return scc' else addSCC v scc' stack

push stack e = modifySTRef' stack (e:)
pop  stack   = do
    e <- head <$> readSTRef stack
    modifySTRef' stack tail
    return e

In [3]:
exG = G.buildG (1, 6) [(1, 2), (1, 3), (2, 4), (5, 6)]
tarjan exG
G.scc exG

[[3],[4],[2],[1],[6],[5]]

[Node {rootLabel = 6, subForest = []},Node {rootLabel = 5, subForest = []},Node {rootLabel = 4, subForest = []},Node {rootLabel = 3, subForest = []},Node {rootLabel = 2, subForest = []},Node {rootLabel = 1, subForest = []}]